# nn.Module로 구현하는 선형 회귀

파이토치에서 이미 구현되어져 있는 함수들을 불러와서 선형 회귀 모델을 구현할 수 있다. 예를 들어 파이토치에서는 선형 회귀 모델이 nn.Linear()라는 함수로, 평균 제곱 오차가 nn.functional.mse_loss()라는 함수로 구현되어져 있다.

```python
import torch.nn as nn
model = nn.Linear(input_dim, output_dim)

import torch.nn.functional as F
cost = F.mse_loss(prediction, y_train)
```

## 단순 선형 회귀 구현

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
torch.manual_seed(1)

$ W = 2, b = 0 $ 임을 알고 데이터 선언

In [3]:
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

선형 회귀 모델 구현
- nn.Linear()는 입력의 차원과 출력의 차원을 인수로 받는다.

In [4]:
input_dim = 1
output_dim = 1
model = nn.Linear(input_dim, output_dim)

In [5]:
x_train.size(), y_train.size()

(torch.Size([3, 1]), torch.Size([3, 1]))

x_train, y_train 모두 1차원이므로 1을 각 인자로 설정한다. model에는 가중치 W와 편향 b가 저장되어 있고 이를 parameters() 함수를 사용해서 불러올 수 있다.

In [6]:
print(list(model.parameters()))

[Parameter containing:
tensor([[0.5153]], requires_grad=True), Parameter containing:
tensor([-0.4414], requires_grad=True)]


첫번째 값이 가중치 W이고 두번째 값이 편향 b이다. 두 값 모두 랜덤한 값으로 초기화되어 있고 학습을 위해 requires_grad 인자가 True 값으로 설정되어있다.

In [7]:
print(list(nn.Linear(3, 2).parameters()))

[Parameter containing:
tensor([[-0.1119,  0.2710, -0.5435],
        [ 0.3462, -0.1188,  0.2937]], requires_grad=True), Parameter containing:
tensor([ 0.0803, -0.0707], requires_grad=True)]


입출력 차원에 따라 가중치와 편향의 크기가 달라지는 것을 알 수 있다.

In [8]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

최적화 알고리즘(옵티마이저)를 정의한다. 앞에서는 직접 가중치와 편향을 리스트로 전달했지만 model의 parameters 함수로 전달할 수 있다. 사용하는 알고리즘은 SGD(경사하강법)이고 학습률은 0.01로 정의했다.

In [9]:
EPOCHS = 2000
for epoch in range(1, EPOCHS+1):
    pred = model(x_train)
    cost = nn.functional.mse_loss(pred, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print(f"epoch: {epoch:4d}/{EPOCHS} | cost: {cost:.8f}")

epoch:  100/2000 | cost: 0.00280407
epoch:  200/2000 | cost: 0.00173274
epoch:  300/2000 | cost: 0.00107072
epoch:  400/2000 | cost: 0.00066164
epoch:  500/2000 | cost: 0.00040886
epoch:  600/2000 | cost: 0.00025265
epoch:  700/2000 | cost: 0.00015612
epoch:  800/2000 | cost: 0.00009647
epoch:  900/2000 | cost: 0.00005961
epoch: 1000/2000 | cost: 0.00003684
epoch: 1100/2000 | cost: 0.00002276
epoch: 1200/2000 | cost: 0.00001407
epoch: 1300/2000 | cost: 0.00000869
epoch: 1400/2000 | cost: 0.00000537
epoch: 1500/2000 | cost: 0.00000332
epoch: 1600/2000 | cost: 0.00000205
epoch: 1700/2000 | cost: 0.00000127
epoch: 1800/2000 | cost: 0.00000078
epoch: 1900/2000 | cost: 0.00000048
epoch: 2000/2000 | cost: 0.00000030


In [10]:
new_var = torch.FloatTensor([[4.0]])
pred_y = model(new_var)
print(f"input: {new_var.item()}, value: {pred_y}")

input: 4.0, value: tensor([[7.9989]], grad_fn=<AddmmBackward0>)


정답 값인 8에 거의 근접하고 있다

In [11]:
print(list(model.parameters()))

[Parameter containing:
tensor([[1.9994]], requires_grad=True), Parameter containing:
tensor([0.0014], requires_grad=True)]


forward/backward 연산
1. forward 연산
    - 값 x로부터 예측된 y를 얻는 것
    - 학습 전 x_train을 통해 예측값을 얻는 것
    - 학습이 완료된 후 new_var를 통해 예측값을 얻는 것
    
2. backward 연산
    - 학습 과정에서 비용 함수를 미분하여 기울기을 구하는 것
    - cost.backward() 함수는 비용 함수로부터 기울기를 구하라는 의미로 backward 연산

## 다중 선형 회귀 구현

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(1)

In [3]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [4]:
model = nn.Linear(3, 1)

In [5]:
print(list(model.parameters()))

[Parameter containing:
tensor([[ 0.2975, -0.2548, -0.1119]], requires_grad=True), Parameter containing:
tensor([0.2710], requires_grad=True)]


In [9]:
optimizer = optim.SGD(model.parameters(), lr=1e-5)

In [11]:
EPOCHS = 2000
for epoch in range(1, EPOCHS+1):
    pred = model(x_train)
    cost = F.mse_loss(pred, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print(f"epoch: {epoch:4d}/EPOCHS | cost: {cost}")

epoch:  100/EPOCHS | cost: 0.22601108253002167
epoch:  200/EPOCHS | cost: 0.22392964363098145
epoch:  300/EPOCHS | cost: 0.22195272147655487
epoch:  400/EPOCHS | cost: 0.22007402777671814
epoch:  500/EPOCHS | cost: 0.21828775107860565
epoch:  600/EPOCHS | cost: 0.21659298241138458
epoch:  700/EPOCHS | cost: 0.21496835350990295
epoch:  800/EPOCHS | cost: 0.21343059837818146
epoch:  900/EPOCHS | cost: 0.21197159588336945
epoch: 1000/EPOCHS | cost: 0.21057219803333282
epoch: 1100/EPOCHS | cost: 0.20924659073352814
epoch: 1200/EPOCHS | cost: 0.20798702538013458
epoch: 1300/EPOCHS | cost: 0.20677998661994934
epoch: 1400/EPOCHS | cost: 0.2056293785572052
epoch: 1500/EPOCHS | cost: 0.2045356035232544
epoch: 1600/EPOCHS | cost: 0.2034899741411209
epoch: 1700/EPOCHS | cost: 0.2024952918291092
epoch: 1800/EPOCHS | cost: 0.20155031979084015
epoch: 1900/EPOCHS | cost: 0.20064163208007812
epoch: 2000/EPOCHS | cost: 0.19978323578834534


# 클래스로 파이토치 모델 구현

## 모델을 클래스로 구현

In [12]:
model = nn.Linear(1, 1)

In [14]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)
        
    def forward(self, x):
        return self.linear(x)

class 로 구현된 모델은 nn.Module을 상속받는다. 

- `__init__()`: 모델의 구조와 동작을 정의하는 생성자를 정의. 파이썬에서 객체가 가지는 속성값을 초기화하는 역할로 객체가 생성될 때 자동으로 호출된다.
- `super()`: 클래스가 nn.Module 클래스의 속성들을 가지고 초기화된다.
- `forward()`: 모델이 학습데이터를 입력받아서 forward 연산을 수행

## 단순 선형 회귀 클래스로 구현

In [26]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)
        
    def forward(self, x):
        return self.linear(x)

In [27]:
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

model = LinearRegressionModel()
optimizer = optim.SGD(model.parameters(), lr=1e-2)

EPOCHS = 2000
for epoch in range(1, EPOCHS+1):
    pred = model(x_train)
    cost = F.mse_loss(pred, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print(f"epoch: {epoch:4d}/{EPOCHS} | cost: {cost}")

epoch:  100/2000 | cost: 0.01705176569521427
epoch:  200/2000 | cost: 0.010536939837038517
epoch:  300/2000 | cost: 0.006511184852570295
epoch:  400/2000 | cost: 0.0040235151536762714
epoch:  500/2000 | cost: 0.002486287383362651
epoch:  600/2000 | cost: 0.001536373165436089
epoch:  700/2000 | cost: 0.0009493827819824219
epoch:  800/2000 | cost: 0.0005866587744094431
epoch:  900/2000 | cost: 0.000362524384399876
epoch: 1000/2000 | cost: 0.00022401548631023616
epoch: 1100/2000 | cost: 0.00013842983753420413
epoch: 1200/2000 | cost: 8.55389007483609e-05
epoch: 1300/2000 | cost: 5.285831866785884e-05
epoch: 1400/2000 | cost: 3.266330895712599e-05
epoch: 1500/2000 | cost: 2.018342274823226e-05
epoch: 1600/2000 | cost: 1.2472162779886276e-05
epoch: 1700/2000 | cost: 7.707370059506502e-06
epoch: 1800/2000 | cost: 4.762485332321376e-06
epoch: 1900/2000 | cost: 2.9432555948005756e-06
epoch: 2000/2000 | cost: 1.819024305405037e-06


## 다중 선형 회귀 클래스로 구현

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [30]:
torch.manual_seed(1)

In [31]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [32]:
class MultiVariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)
        
    def forward(self, x):
        return self.linear(x)

In [33]:
model = MultiVariateLinearRegressionModel()
optimizer = optim.SGD(model.parameters(), lr=1e-5)

In [35]:
EPOCHS = 2000
for epoch in range(1, EPOCHS+1):
    pred = model(x_train)
    cost = F.mse_loss(pred, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print(f"epoch: {epoch:4d}/{EPOCHS} | cost: {cost}")

epoch:  100/2000 | cost: 0.22601108253002167
epoch:  200/2000 | cost: 0.22392964363098145
epoch:  300/2000 | cost: 0.22195272147655487
epoch:  400/2000 | cost: 0.22007402777671814
epoch:  500/2000 | cost: 0.21828775107860565
epoch:  600/2000 | cost: 0.21659298241138458
epoch:  700/2000 | cost: 0.21496835350990295
epoch:  800/2000 | cost: 0.21343059837818146
epoch:  900/2000 | cost: 0.21197159588336945
epoch: 1000/2000 | cost: 0.21057219803333282
epoch: 1100/2000 | cost: 0.20924659073352814
epoch: 1200/2000 | cost: 0.20798702538013458
epoch: 1300/2000 | cost: 0.20677998661994934
epoch: 1400/2000 | cost: 0.2056293785572052
epoch: 1500/2000 | cost: 0.2045356035232544
epoch: 1600/2000 | cost: 0.2034899741411209
epoch: 1700/2000 | cost: 0.2024952918291092
epoch: 1800/2000 | cost: 0.20155031979084015
epoch: 1900/2000 | cost: 0.20064163208007812
epoch: 2000/2000 | cost: 0.19978323578834534


# 미니 배치와 데이터 로드

## 미니 배치와 배치 크기

다중 선형 회귀에서 데이터 샘플 5개를 하나의 행렬로 선언하여 학습하였다. 만약 현업에서도 이런 식으로 수행한다면 데이터의 양이 너무 방대해서 비효율적일 수 있고 메모리의 한계로 계산이 불가능한 경우도 있다.

그렇기 때문에 전체 데이터를 작은 단위로 나누어서 단위별로 학습하는 개념이 나왔고 이러한 단위를 **미니 배치**라고 한다

전체 데이터를 100개 미니 배치 단위를 10개라고 하면 데이터를 10개씩 가져가서 비용을 계산하고 경사 하강법을 수행한다. 또 다음 미니 배치(10개)를 가져가서 경사하강법을 수행한다. 이렇게 총 10번, 전체 데이터 100개에 대해 1회 수행하면 1 epoch가 끝나게 된다.

- **배치 경사 하강법**

전체 데이터에 대해서 한 번에 경사 하강법을 수행하는 방법으로, 전체 데이터를 한번에 사용하기 때문에 최적값에 수렴하는 과정이 매우 안정적이지만 계산량이 많이 소요된다

- **미니 배치 경사 하강법**

미니 배치 단위로 경사 하강법을 수행하는 방법으로, 데이터의 일부만으로 수행하기 때문에 최적값에 수렴하는 과정이 배치 경사 하강법에 비해 불안정하지만 훈련 속도가 빠르다


배치의 크기는 보통 2의 제곱수를 사용하는데 그 이유는 CPU와 GPU의 메모리가 2의 배수이기 때문에 배치의 크기가 2의 제곱수일 경우 데이터 송수신의 효율을 높일 수 있다.

## 이터레이션

이터레이션은 1 epoch를 수행할 때 가중치가 업데이트되는 횟수이다.
위의 예로 전체 데이터가 100개이고 배치 크기가 10이라고 가정하면 이터레이션은 10이다.

## 데이터 로드

파이토치에서는 데이터를 쉽게 다룰 수 있는 도구로 Dataset, DataLoader를 제공한다.
이를 사용해서 미니 배치 학습, 데이터 셔플, 병렬 처리까지 수행 가능하다.
기본적인 사용 방법은 Dataset을 정의하고 이를 DataLoader에 전달하는 것이다.

Dataset을 커스텀해서 만들 수도 있고 텐서를 입력받아 Dataset의 형태로 바꾸어 사용할 수도 있다.

In [37]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [38]:
from torch.utils.data import TensorDataset # Tensor -> Dataset
from torch.utils.data import DataLoader

TensorDataset은 기본적으로 Tensor를 입력값으로 받는다.

In [39]:
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  90], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

In [40]:
dataset = TensorDataset(x_train, y_train)

정의한 Tensor를 TensorDataset의 입력으로 사용해서 dataset 변수에 저장한다.

In [41]:
dataset

파이토치 데이터셋을 만든 후에는 데이터로더를 사용할 수 있다. 데이터로더는 기본적으로 데이터셋, 미니 배치의 크기 2개의 인자를 입력받는다. 이 때 미니 배치의 크기는 2의 배수를 통상적으로 사용한다. 이 외에도 많이 사용되는 인자 shuffle이 있는데 이를 True로 설정하면 Epoch마다 데이터셋을 섞어서 데이터를 학습할 수 있다

In [42]:
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [43]:
model = nn.Linear(3, 1)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

In [44]:
EPOCHS = 20
for epoch in range(1, EPOCHS+1):
    for batch_idx, samples in enumerate(dataloader):
        x_train, y_train = samples
        pred = model(x_train)
        
        cost = F.mse_loss(pred, y_train)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        print(f"Epoch: {epoch:4d}/EPOCHS | Batch: {batch_idx+1}/{len(dataloader)} | Cost: {cost}")

Epoch:    1/EPOCHS | Batch: 1/3 | Cost: 40394.078125
Epoch:    1/EPOCHS | Batch: 2/3 | Cost: 10625.849609375
Epoch:    1/EPOCHS | Batch: 3/3 | Cost: 5451.56982421875
Epoch:    2/EPOCHS | Batch: 1/3 | Cost: 937.502685546875
Epoch:    2/EPOCHS | Batch: 2/3 | Cost: 265.94476318359375
Epoch:    2/EPOCHS | Batch: 3/3 | Cost: 244.42251586914062
Epoch:    3/EPOCHS | Batch: 1/3 | Cost: 3.5231642723083496
Epoch:    3/EPOCHS | Batch: 2/3 | Cost: 55.838958740234375
Epoch:    3/EPOCHS | Batch: 3/3 | Cost: 2.3711564540863037
Epoch:    4/EPOCHS | Batch: 1/3 | Cost: 21.61503791809082
Epoch:    4/EPOCHS | Batch: 2/3 | Cost: 1.5361409187316895
Epoch:    4/EPOCHS | Batch: 3/3 | Cost: 23.620941162109375
Epoch:    5/EPOCHS | Batch: 1/3 | Cost: 12.848388671875
Epoch:    5/EPOCHS | Batch: 2/3 | Cost: 17.57853126525879
Epoch:    5/EPOCHS | Batch: 3/3 | Cost: 18.382184982299805
Epoch:    6/EPOCHS | Batch: 1/3 | Cost: 11.40179443359375
Epoch:    6/EPOCHS | Batch: 2/3 | Cost: 20.960948944091797
Epoch:    6/EPOC

배치마다 차이가 있지만 기본적으로 epoch가 증가함에 따라 cost값이 감소하고 있다.

In [46]:
new_var = torch.FloatTensor([67, 55, 94])
y_pred = model(new_var)
y_pred.item()

140.9924774169922

임의의 데이터 샘플 1개를 넣어서 나오는 예측값을 확인해볼 수 있다.

# 커스텀 데이터셋

직접 데이터셋을 만드는 경우 Dataset을 상속받아서 수행할 수 있다. Dataset은 파이토치에서 데이터셋을 제공하는 추상 클래스로 이를 상속받아 아래의 메소드들을 오버라이드하여 커스텀 데이터셋을 만들 수 있다.

커스텀 데이터셋을 만들 때 가장 기본적인 뼈대는 다음과 같고 여기서 가장 기본적인 define은 3개이다.

```python
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        
    def __len__(self):
        
    def __getitem__(self, idx):
```